In [12]:
%matplotlib inline
import astropy.io.fits as pyfits
import numpy as np
from astropy.utils.data import get_pkg_data_filename
import subprocess as sp
import scipy.ndimage as sci_nd
import glob
import matplotlib.pyplot as mpl
from astropy.table import Table, hstack, join
import sys
import os
from astropy.io import ascii
from astropy.table import Column

In [13]:
def read_results_file(fname):
    r""" Reads a galfit HDU cube to get the results for the best fit model,
    which are stored in the header of the cube extension 2. It is assumed that
    a single sersic model was fit.
    Parameters
    ----------
    fname : str
        The name of the result file to read the model parameters from.
    Returns
    -------
    xc : str
        The model X center and its error (separated by whitespace)
    yc : str
        The model Y center and its error (separated by whitespace)
    M : str
        The model magnitude and its error (separated by whitespace)
    R : str
        The model effective radius and its error (separated by whitespace)
    N : str
        The model sersic index and its error (separated by whitespace)
    Q : str
        The model axis ratio and its error (separated by whitespace)
    T : str
        The model position angle and its error (separated by whitespace)
    chi : str
        The reduced chi square value of the fit
    F : str
        A galfit flag indicating the final status of the fit. If 0 everything
        is ok. If 1 it means that GALFIT finished 100 iterations without
        converging. If 2, it means GALFIT results diverged and they cannot
        be trusted,
    References
    ----------
    Examples
    --------
    """
    try:
        hdu=pyfits.open(fname)
        chi=hdu[2].header['CHI2NU']
        xc=hdu[2].header['1_XC'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        yc=hdu[2].header['1_YC'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        M=hdu[2].header['1_MAG'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        R=hdu[2].header['1_RE'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        N=hdu[2].header['1_N'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        Q=hdu[2].header['1_AR'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        T=hdu[2].header['1_PA'].replace('[','').replace('*','').replace('+/-','').replace(']',' -99.00')
        F= hdu[2].header['FLAGS'].replace(' ',',')
        if ('1' in F.split(',')):
            F=1
        elif ('2' in F.split(',')):
            F=2
        else:
            F=0
    except IOError:
        xc,yc,M,R,N,Q,T,chi,F="-99 -99","-99 -99","-99 -99","-99 -99","-99 -99","-99 -99","-99 -99","-99","-99"
    return xc,yc,M,R,N,Q,T,chi,F

In [14]:
masterdata = ascii.read('/run/media/rowland/PURPLEDRIVE/mastertable')
#masterdata = ascii.read('/data1/rowland/mastertable')

#masterdata['NUMBER_F850']

# Matches all Kabooms to masterdata

In [16]:
#KEEP - 1 of 4
#gets which galfit runs went kaboom

filters = ["F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP"]

kaboomis = []
kaboomfs = []
totalrealkaboomn =[]
totalrealkaboomf =[]
#testkaboom =[]
totalgalfitruns = []
totalkaboomN = []
totalkaboomF = []

for filter in filters:
    if filter == 'F850LP':
        tfilter = filter[:-2]
        detect_nums = np.arange(0,len(masterdata[~masterdata['X_IMAGE_{}'.format(tfilter)].mask]))
    else:
        tfilter = filter[:-1]
        detect_nums = np.arange(0,len(masterdata[~masterdata['X_IMAGE_{}'.format(tfilter)].mask]))
    for num in detect_nums:
            totalgalfitruns.append(num)
            #print('num:',num,'filter',filter)
            xc,yc,M,R,N,Q,T,chi,F = read_results_file('/run/media/rowland/PURPLEDRIVE/{}/Outputs/output{}.fits'.format(filter,num))
            #xc,yc,M,R,N,Q,T,chi,F = read_results_file('/data1/rowland/{}W/Outputs/output{}.fits'.format(filter,num))

            if F ==2:
                print(num,filter,'has suspected numerical converence error in current solution')
                totalkaboomN.append(num)
                totalkaboomF.append(filter)
                kaboomfs.append(filter)
                kaboomis.append(num)
                #testkaboom.append(({filter},num))
            #elif F == 0: #good runs
             #   print(num,filter)
            elif F == "-99":
                print(num,filter, 'went Kaboom')
                totalkaboomN.append(num)
                totalkaboomF.append(filter)
                totalrealkaboomn.append(num)
                totalrealkaboomf.append(filter)
            elif F ==1:
                print(num,filter,'error of ususally max #of iterations reached, but sometimes header/psf/image missing')
            else:
                continue
print(np.shape(kaboomis),'errors, plus',np.shape(totalrealkaboomn),'kabooms, out of ',np.shape(totalgalfitruns),'galfit runs of individual detections')
print(np.shape(totalkaboomN),np.shape(totalkaboomF)) #check this!!!!!!

1 F105W has suspected numerical converence error in current solution
4 F105W has suspected numerical converence error in current solution
5 F105W has suspected numerical converence error in current solution
6 F105W has suspected numerical converence error in current solution
7 F105W has suspected numerical converence error in current solution
8 F105W has suspected numerical converence error in current solution
9 F105W has suspected numerical converence error in current solution
10 F105W has suspected numerical converence error in current solution
12 F105W has suspected numerical converence error in current solution
14 F105W has suspected numerical converence error in current solution
15 F105W has suspected numerical converence error in current solution
16 F105W has suspected numerical converence error in current solution
19 F105W has suspected numerical converence error in current solution
20 F105W has suspected numerical converence error in current solution
21 F105W has suspected nume

55 F625W has suspected numerical converence error in current solution
62 F625W has suspected numerical converence error in current solution
67 F625W has suspected numerical converence error in current solution
68 F625W has suspected numerical converence error in current solution
70 F625W has suspected numerical converence error in current solution
73 F625W has suspected numerical converence error in current solution
1 F775W has suspected numerical converence error in current solution
3 F775W has suspected numerical converence error in current solution
5 F775W has suspected numerical converence error in current solution
6 F775W has suspected numerical converence error in current solution
8 F775W has suspected numerical converence error in current solution
17 F775W has suspected numerical converence error in current solution
20 F775W has suspected numerical converence error in current solution
27 F775W went Kaboom
30 F775W has suspected numerical converence error in current solution
42 F

In [5]:
#KEEP -2 of 4
#splits kabooms by filter and sets up next cell

kaboom = Table([kaboomfs,kaboomis],names=('filter','detection_number'))
master = np.arange(0,len(masterdata))

T105 = kaboom[np.where(kaboom['filter']==filters[0])]
T125 = kaboom[np.where(kaboom['filter']==filters[1])]
T140 = kaboom[np.where(kaboom['filter']==filters[2])]
T160 = kaboom[np.where(kaboom['filter']==filters[3])]
T435 = kaboom[np.where(kaboom['filter']==filters[4])]
T606 = kaboom[np.where(kaboom['filter']==filters[5])]
T625 = kaboom[np.where(kaboom['filter']==filters[6])]
T775 = kaboom[np.where(kaboom['filter']==filters[7])]
T814 = kaboom[np.where(kaboom['filter']==filters[8])]
T850L = kaboom[np.where(kaboom['filter']==filters[9])]

mytables = [T105,T125,T140,T160,T435,T606,T625,T775,T814,T850L]

#added here
# realkb = Table([totalrealkaboomn,totalrealkaboomf], names=('detection_number', 'filter'))
# T105kb = realkb[np.where(realkb['filter']==filters[0])]
# T125kb = realkb[np.where(realkb['filter']==filters[1])]
# T140kb = realkb[np.where(realkb['filter']==filters[2])]
# T160kb = realkb[np.where(realkb['filter']==filters[3])]
# T435kb = realkb[np.where(realkb['filter']==filters[4])]
# T606kb = realkb[np.where(realkb['filter']==filters[5])]
# T625kb = realkb[np.where(realkb['filter']==filters[6])]
# T775kb = realkb[np.where(realkb['filter']==filters[7])]
# T814kb = realkb[np.where(realkb['filter']==filters[8])]
# T850Lkb = realkb[np.where(realkb['filter']==filters[9])]
# mytableskb = [T105kb,T125kb,T140kb,T160kb,T435kb,T606kb,T625kb,T775kb,T814kb,T850Lkb]


print(len(np.where(masterdata['NUMBER_F435']>0)[0])) 
print(len(T105),len(T125),len(T140),len(T160),len(T435),len(T606),len(T625),len(T775),len(T814),len(T850L))
type(mytables[0]['detection_number'].astype('int'))

8
35 24 18 21 5 13 21 15 41 16


astropy.table.column.Column

In [6]:
#KEEP -3 of 4
#doing all below in one cycle--WORKS

masks=[]
allfilters=[]
mastermatch = []
idx=[]

#matches kaboom index (in detection order) with masterdata index (that includes masked nondetections)
for i,f in enumerate(filters):
    if f == 'F850LP':
        fn = f[:-2]
        tn = f[1:-2]
    else:
        fn = f[:-1]
        tn = f[1:-1]
    arr1 = (np.where(masterdata['NUMBER_{}'.format(fn)] > 0)[0])
    arr2 = mytables[i]['detection_number'].astype('int')
    newarr = np.zeros(len(arr1))
    newarr[:] = np.nan
    newarr[arr2] = arr1[arr2]
    masks.append(newarr)
for mask in masks:
    eachfilter = mask[~np.isnan(mask)]
    allfilters.append(eachfilter)
for i in allfilters:
    arr3 = np.arange(0,len(masterdata),1) 
    arr4 = i.astype('int')
    newerarr = np.zeros(len(arr3))
    newerarr[:] = np.nan
    newerarr[arr4] = arr3[arr4]
    mastermatch.append(newerarr)
    
#gets index when there was never a z match per filter
for filter in filters:
    if filter == 'F850LP':
        tfilter = filter[:-2]
        indices = np.where(masterdata['X_IMAGE_{}'.format(tfilter)].mask)
        idx.append(indices)
    else:
        tfilter = filter[:-1]
        indices = np.where(masterdata['X_IMAGE_{}'.format(tfilter)].mask)
        idx.append(indices)
        
#creates 0 & 1 table of galfit runs for kaboom or not
mastermatch = np.array(mastermatch)
print('shape:',np.shape(mastermatch))
mastermatch[np.where(mastermatch>=0.0)]=1 #1 means had a kaboom
mastermatch[np.where(mastermatch!=1.)]=0 #0 means no kaboom

#makes nongalfit runs into nan values
for i,master in enumerate(mastermatch):
    master[idx[i]]=np.nan
    
#gets total num of kabooms per detection, another column to add
sums = np.nansum(mastermatch,axis=0).astype('int') 

shape: (10, 89)


/software/local/lib64/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater_equal


In [7]:
#KEEP -4 of 4
#final table of galfit results #nan=no galfit run; 0=good; 1=KABOOM

masterkabooms = Table([masterdata['recno'],mastermatch[0],mastermatch[1],mastermatch[2],mastermatch[3],mastermatch[4],\
                       mastermatch[5],mastermatch[6],mastermatch[7],mastermatch[8],mastermatch[9],\
                      sums],names=("masterdataRECNO","F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP","KaboomTotals"))
masterkabooms

masterdataRECNO,F105W,F125W,F140W,F160W,F435W,F606W,F625W,F775W,F814W,F850LP,KaboomTotals
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,1.0,1.0,1.0,nan,0.0,1.0,1.0,1.0,0.0,7
3,0.0,0.0,0.0,0.0,nan,0.0,0.0,0.0,1.0,0.0,1
4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,5
5,1.0,0.0,0.0,0.0,nan,0.0,0.0,0.0,1.0,0.0,2
6,nan,nan,nan,nan,nan,nan,nan,1.0,nan,nan,1
7,1.0,1.0,1.0,1.0,nan,0.0,1.0,1.0,0.0,0.0,6
8,1.0,1.0,0.0,0.0,nan,0.0,0.0,0.0,nan,0.0,2
9,1.0,0.0,0.0,0.0,nan,0.0,0.0,1.0,1.0,0.0,3


# trying to amend code for real kabooms

In [ ]:
#KEEP 2 of 4


realkb = Table([totalkaboomN,totalkaboomF], names=('detection_number', 'filter'))
T105kb = realkb[np.where(realkb['filter']==filters[0])]
T125kb = realkb[np.where(realkb['filter']==filters[1])]
T140kb = realkb[np.where(realkb['filter']==filters[2])]
T160kb = realkb[np.where(realkb['filter']==filters[3])]
T435kb = realkb[np.where(realkb['filter']==filters[4])]
T606kb = realkb[np.where(realkb['filter']==filters[5])]
T625kb = realkb[np.where(realkb['filter']==filters[6])]
T775kb = realkb[np.where(realkb['filter']==filters[7])]
T814kb = realkb[np.where(realkb['filter']==filters[8])]
T850Lkb = realkb[np.where(realkb['filter']==filters[9])]
mytableskb = [T105kb,T125kb,T140kb,T160kb,T435kb,T606kb,T625kb,T775kb,T814kb,T850Lkb]

In [6]:
#KEEP -3 of 4
#doing all below in one cycle--WORKS

masks=[]
allfilters=[]
mastermatch = []
idx=[]

#matches kaboom index (in detection order) with masterdata index (that includes masked nondetections)
for i,f in enumerate(filters):
    if f == 'F850LP':
        fn = f[:-2]
        tn = f[1:-2]
    else:
        fn = f[:-1]
        tn = f[1:-1]
    arr1 = (np.where(masterdata['NUMBER_{}'.format(fn)] > 0)[0])
    
#need to add in here but keep flags...maybe not bc will need to rerun all regardless of F=2 or ='-99' 
#didnt keep flags just did all b/c need to rerun both flags
    arr2 = mytableskb[i]['detection_number'].astype('int')
    newarr = np.zeros(len(arr1))
    newarr[:] = np.nan
    newarr[arr2] = arr1[arr2]
    masks.append(newarr)
for mask in masks:
    eachfilter = mask[~np.isnan(mask)]
    allfilters.append(eachfilter)
for i in allfilters:
    arr3 = np.arange(0,len(masterdata),1) 
    arr4 = i.astype('int')
    newerarr = np.zeros(len(arr3))
    newerarr[:] = np.nan
    newerarr[arr4] = arr3[arr4]
    mastermatch.append(newerarr)
    
#gets index when there was never a z match per filter
for filter in filters:
    if filter == 'F850LP':
        tfilter = filter[:-2]
        indices = np.where(masterdata['X_IMAGE_{}'.format(tfilter)].mask)
        idx.append(indices)
    else:
        tfilter = filter[:-1]
        indices = np.where(masterdata['X_IMAGE_{}'.format(tfilter)].mask)
        idx.append(indices)
        
        
#trying add here---not here
# for filter in filters:
#     if filter == 'F850LP':
#         tfilter = filter[:-2]
#         indices2 = 
#         idx2.append(indices2)
#     else:
#         tfilter = filter[:-1]
#         indices2 = 
#         idx2.append(indices2)
       
    
    
#creates 0 & 1 table of galfit runs for kaboom or not
mastermatch = np.array(mastermatch)
print('shape:',np.shape(mastermatch))
mastermatch[np.where(mastermatch>=0.0)]=1 #1 means had a kaboom
mastermatch[np.where(mastermatch!=1.)]=0 #0 means no kaboom

#makes nongalfit runs into nan values
for i,master in enumerate(mastermatch):
    master[idx[i]]=np.nan

    
#add diff flag here
    
    
#gets total num of kabooms per detection, another column to add
sums = np.nansum(mastermatch,axis=0).astype('int') 

recno NUMBER_F850
----- -----------
    1         606
recno NUMBER_F850
----- -----------
    2         653
recno NUMBER_F850
----- -----------
    3         948
recno NUMBER_F850
----- -----------
    4         645
recno NUMBER_F850
----- -----------
    5         430
recno NUMBER_F850
----- -----------
    6          --
recno NUMBER_F850
----- -----------
    7         462
recno NUMBER_F850
----- -----------
    8         989
recno NUMBER_F850
----- -----------
    9         766
recno NUMBER_F850
----- -----------
   10         597
recno NUMBER_F850
----- -----------
   11         707
recno NUMBER_F850
----- -----------
   12         450
recno NUMBER_F850
----- -----------
   13         421
recno NUMBER_F850
----- -----------
   14         892
recno NUMBER_F850
----- -----------
   15         587
recno NUMBER_F850
----- -----------
   16         244
recno NUMBER_F850
----- -----------
   17        1010
recno NUMBER_F850
----- -----------
   18        1051
recno NUMBER_F850
----- ----

In [ ]:
masterkabooms = Table([masterdata['recno'],mastermatch[0],mastermatch[1],mastermatch[2],mastermatch[3],mastermatch[4],\
                       mastermatch[5],mastermatch[6],mastermatch[7],mastermatch[8],mastermatch[9],\
                      sums],names=("recno","F105W_kabooms","F125W_kabooms","F140W_kabooms","F160W_kabooms","F435W_kabooms","F606W_kabooms","F625W_kabooms","F775W_kabooms","F814W_kabooms","F850LP_kabooms","KaboomTotals"))
masterkabooms

In [12]:
len(masterkabooms[np.where(masterkabooms['KaboomTotals']==0.0)]) 
#number of galaxies with no kabooms but idk how many had detections to begin with

20

In [ ]:
#merge kabooms with masterdata




# Rerunning Kaboom detections with amended params

In [11]:
#Bruno's github codes

def get_fixpars_default():
    r""" Returns the default dictionary containing the information on whether
    or not to fix any parameter of the fit. By default, all parameters are
    not fixed.
    Parameters
    ----------
    Returns
    -------
    fixpars : dict
        A dictionary for each of the sersic parameters setting the fix/free key.
    References
    ----------
    Examples
    --------
    """
    return {'x':1,'y':1,'m':1,'re':1,'n':0,'q':1,'pa':1,'sky':1} #AMENDED HERE TO ALLOW n TO VARY


def write_object(model,x,y,m,re,n,ba,pa,num,fixpars=None):
    r""" Returns a string object containing a general description for a galaxy
    profile model in GALFIT, with the input parameters as first guesses.
    Parameters
    ----------
    model : str
    Returns
    -------
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    objString = ""
    objString += "#Object number: %i\n"%(num)
    objString += " 0) %s             # Object type\n"%(model)
    objString += " 1) %6.4f %6.4f  %i %i    # position x, y        [pixel]\n"%(x,y,fixpars['x'],fixpars['y'])
    objString += " 3) %4.4f      %i       # total magnitude\n"%(m,fixpars['m'])
    objString += " 4) %4.4f       %i       #     R_e              [Pixels]\n"%(re,fixpars['re'])
    objString += " 5) %4.4f       %i       # Sersic exponent (deVauc=4, expdisk=1)\n"%(n,fixpars['n'])
    objString += " 9) %4.4f       %i       # axis ratio (b/a)   \n"%(ba,fixpars['q'])
    objString += "10) %4.4f       %i       # position angle (PA)  [Degrees: Up=0, Left=90]\n"%(pa,fixpars['pa'])
    objString += " Z) 0                  #  Skip this model in output image?  (yes=1, no=0)\n"
    objString += " \n"
    return objString


def input_file(f,modelsString,magzpt,sky,x_range,y_range,sconvbox,pixscale,imgname='input.fits',outname="output.fits",psfname='none',maskname="none",signame='none',fixpars=None):
    r""" Writes an input galfit file with the provided parameters.
    Parameters
    ----------
    f : file pointer
        Python file pointer to be written
    modelsString : str
        A single string containing the starting parameters of the models to be
        used. Model strings can be generated by the write_object function.
    magzpt : float
        Magnitude zeropoint of the image data. Needed for correct model
        magnitudes.
    sky : float
        Initial value for sky background (assumed to be constant)
    x_range : tuple, int
        X-axis boundaries to be used in image fitting (useful to fit a subset
        of a larger image).
    y_range : tuple, int
        Same as z_range, but for y-axis boundaries.
    sconvbox : int
        Size of the convolution box to be applied to the model image (should be
        at least the size of the PSF image).
    pixscale : float
        Pixel scale (in arcsecond/pixel) of the input image data
    imgname : str, optional
        Name of the image with the input data (default - input.fits)
    outname : str, optional
        Name of the output image block to be created by GALFIT
        (default - output.fits)
    psfname : str, optional
        Name of the psf fits file, to be used for model convolution
        (default - none, no psf used)
    maskname : str, optional
        Name of the mask file to be used in the fit. It mask bad pixel values
        and others (e.g. neighbor objects) that are ignored by the fitting
        algorithm (default - none, no mask used).
    signame : str, optional
        Error image corresponding to the input data.
        (default - none, sigma image created internally).
    fixpars : dict
        Dictionary which controls if parameters are fixed or not for the fit.
        This is used here to control wether the sky background is fixed or a
        free parameter for the fit.
    Returns
    -------
        None
        It writes to the file "f" and returns nothing.
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    assert len(x_range)==len(y_range)==2,"x_range,y_range must have two elements"
    assert x_range[1]>x_range[0],"x_range must be sorted in ascendent order"
    assert y_range[1]>y_range[0],"y_range must be sorted in ascendent order"

    f.write("================================================================================\n")
    f.write("# IMAGE and GALFIT CONTROL PARAMETERS\n")
    f.write("A) %s         # Input data image (FITS file)\n"%imgname)
    f.write("B) %s        # Output data image block\n"%outname)
    f.write("C) %s                # Sigma image name (made from data if blank or 'none' \n"%signame)
    f.write("D) %s         # Input PSF image and (optional) diffusion kernel\n"%psfname)
    f.write("E) 1                   # PSF fine sampling factor relative to data \n")
    f.write("F) %s                # Bad pixel mask (FITS image or ASCII coord list)\n"%maskname)
    f.write("G) none                # File with parameter constraints (ASCII file) \n")
    f.write("H) %i    %i   %i    %i # Image region to fit (xmin xmax ymin ymax)\n"%(x_range[0],x_range[1],y_range[0],y_range[1]))
    f.write("I) %i    %i          # Size of the convolution box (x y)\n"%(sconvbox,sconvbox))
    f.write("J) %7.5f             # Magnitude photometric zeropoint \n"%magzpt)
    f.write("K) %.3f %.3f        # Plate scale (dx dy)   [arcsec per pixel]\n"%(pixscale,pixscale))
    f.write("O) regular             # Display type (regular, curses, both)\n")
    f.write("P) 0                   # Options: 0=normal run; 1,2=make model/imgblock and quit\n")
    f.write("\n")
    f.write("# INITIAL FITTING PARAMETERS\n")
    f.write("#\n")
    f.write("#For object type, the allowed functions are:\n")
    f.write("#nuker, sersic, expdisk, devauc, king, psf, gaussian, moffat,\n")
    f.write("#ferrer, and sky.\n")
    f.write("#\n")
    f.write("#Hidden parameters will only appear when theyre specified:\n")
    f.write("#C0 (diskyness/boxyness),\n")
    f.write("#Fn (n=integer, Azimuthal Fourier Modes).\n")
    f.write("#R0-R10 (PA rotation, for creating spiral structures).\n")
    f.write("#\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("#  par)    par value(s)    fit toggle(s)   parameter description\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("\n")

    f.write(modelsString)

    f.write("# Object: Sky\n")
    f.write(" 0) sky                    #  object type\n")
    f.write(" 1) %7.4f      %i          #  sky background at center of fitting region [ADUs]\n"%(sky,fixpars['sky']))
    f.write(" 2) 0.0000      0          #  dsky/dx (sky gradient in x)\n")
    f.write(" 3) 0.0000      0          #  dsky/dy (sky gradient in y)\n")
    f.write(" Z) 0                      #  output option (0 = resid., 1 = Dont subtract)")
    f.close()
    return


#amended code

def select_object_segmap(segmap,index):
    Regions = np.zeros_like(segmap)
    if index not in segmap:
        raise ValueError('index not in segmentation map')
    Regions[segmap==index]=index
    return Regions

In [ ]:
#change this to grab data from master kabooms....but also need to merge tables before this


#collecting data of detections i want galfit to fit (ie cluster galaxies)
#WORKS


model = 'sersic' #from bruno
object_numbers = []
xpixs = []

def modString(filter):
    modelsList = [] #FOR INDIVIDUAL DETECTIONS
    modelsString = []   
    xs = [] 
    ys = []
    print(filter,len(masterdata[~masterdata['X_IMAGE_{}'.format(filter)].mask]))
    i = 1
    for detection in masterdata[~masterdata['X_IMAGE_{}'.format(filter)].mask]: #to only run on subset for testing
        ## DEFINING PARAMETERS
        try:
            if not detection['X_IMAGE_{}'.format(filter)] == np.nan:
                
                num = i #detection['NUMBER_{}'.format(filter)] #object detection number from SEx list
                i += 1
                ba = 1 - detection['ELLIPTICITY_{}'.format(filter)] 
                pa = 90 - detection['THETA_IMAGE_{}'.format(filter)] 
                x = detection['X_IMAGE_{}'.format(filter)] 
                y = detection['Y_IMAGE_{}'.format(filter)]
                re = detection['FLUX_RADIUS_{}'.format(filter)]
                m = detection['MAG_AUTO_{}'.format(filter)] #+2.5*np.log10(100)
                n = 4     #this is sersic index, 4 b/c mostly expecting distinct cores & extended disks
                list_of_strings = write_object(model,x,y,m,re,n,ba,pa,num,fixpars=None) 
                object_numbers.append(num)
                xs.append(x) #so i can slice these the same as modelsList in next cell
                ys.append(y)
                modelsList.append(list_of_strings) #FOR INDIVIDUAL DETECTIONS INPUT FILES
                modelsString = ''.join(modelsList) #FOR ALL DETECTIONS INPUT FILE
            else:
                continue
        except np.ma.MaskError:
            continue
    return modelsList, modelsString, object_numbers, xs,ys

# Getting full cluster models

In [6]:
for filter in filters:
    if filter == 'F850LP':
        tfilter = filter[:-2]
        detect_nums = np.arange(0,len(masterdata[~masterdata['X_IMAGE_{}'.format(tfilter)].mask]))
    else:
        tfilter = filter[:-1]
        detect_nums = np.arange(0,len(masterdata[~masterdata['X_IMAGE_{}'.format(tfilter)].mask]))  
        print(np.shape(detect_nums))
        for count in detect_nums:
            image = get_pkg_data_filename('/data1/rowland/{}/Outputs/output{}.fits'.format(filter,count))
            image = get_pkg_data_filename('/data1/rowland/{}/Outputs/output{}.fits'.format(filter,count))
            image_2 = pyfits.getdata(image, ext=2) #model

    #models = 

(53,)
(53,)
(53,)
(53,)


URLError: <urlopen error Failed to download /data1/rowland/F160W/Outputs/output46.fits from the following repositories:

  - http://data.astropy.org/
  - http://www.astropy.org/astropy-data/>

# TEST CELLS BELOW

In [13]:
 #how many filters had kabooms per galaxy
    
    
# sums = np.nansum(mastermatch,axis=0).astype('int') #this wont let me see the first row b/c values =0 & nans become 0
# numbadfilters =[]
# for count,s in enumerate(sums):
#     if count == 0:
#         count = 99
#     else:
#         count = count
#     #print('count',count)
#     howmanyfilters = s/count
#     numbadfilters.append(howmanyfilters)
#     #print(count,howmanyfilters)
# #print(np.shape(numbadfilters)) #this is per detection row in masterdata, the number of filters that went kaboom    
# numbadfilters = np.array(numbadfilters)
# numbadfilters[np.where(numbadfilters==0.0)]=np.nan
# for count,i in enumerate(numbadfilters):
#     print(count,i)

In [14]:
# print(numbadfilters)

In [15]:
# print(type(numbadfilters))
# numbadfilters=np.array(numbadfilters)
# numbers = np.where(numbadfilters==0.0)
# print('nums:',numbers)
# numbadfilters[np.where(numbadfilters==0.0)]=np.nan
# numbadfilters

In [16]:
# totals = []
# for count,row in enumerate(masterkabooms):
#     for value in row:
#         if value >0:
#         #if value != np.nan:
#             rownum = count
#             totals.append(count)
#             print(count)
#         #print(value, type(value))
#     #print(row =='int')
#     #print(row.as_void()) #prints all rows

In [17]:
# seen = set()
# uniq = []
# for row in masterkabooms:
#     #for value in row:
#     for x in row:
#         if x not in seen:
#             uniq.append(x)
#             seen.add(x)

In [18]:
# uniq = np.array(uniq)
# uniq.astype('int')
# finalis = uniq[~np.isnan(uniq)]
# print(len(finalis),finalis)

In [19]:
#type(uniq),np.shape(uniq),uniq


#### does it all in single steps tho

In [20]:
#does everything in one cycle to create a mask per each filters total detections, not masterdata yet

# masks=[]

# for i,f in enumerate(filters):
#     if f == 'F850LP':
#         fn = f[:-2]
#         tn = f[1:-2]
#     else:
#         fn = f[:-1]
#         tn = f[1:-1]
#     arr1 = (np.where(masterdata['NUMBER_{}'.format(fn)] > 0)[0])
#     print(len(arr1)) #correct
#     arr2 = mytables[i]['detection_number'].astype('int')
#     newarr = np.zeros(len(arr1))
#     newarr[:] = np.nan
#     newarr[arr2] = arr1[arr2]
#     masks.append(newarr)


In [21]:
# fullt = np.zeros((10,89))
# #print(np.shape(fullt),fullt)
# for filternum,mask in enumerate(masks):
#     for count,val in enumerate(mask):
#         print(val)
#         print(fullt[filternum][val])
#         fullt[filternum] = fullt[filternum][val] + mask[count]
# print(np.shape(fullt),fullt)


In [22]:
#dont need anymore

# #testing how to match masks to masterdata size and then insert it into above loop before masks.append
# import numpy.ma as ma
# mastermatch = []
# for i in masks:
#     arr3 = np.arange(0,len(masterdata),1) #might wana start at 0 and +1 b/c recno goes from 1-89 but 4 now this is 0-88
#     #arrtest = np.ma(i)
#     print('masterdata shape:',np.shape(arr3),type(arr3))
#     arr4 = np.ma(i)
#     print('i type:',type(i))
#     ##i.compressed()
#     ##arr4 = i#.compressed()
#     print('checking original mask array:',type(arr4),np.shape(arr4),arr4.dtype,arr4)
#     newerarr = np.zeros(len(arr3))
#     newerarr[:] = np.nan
#     ##arr5=[]
#     ##print('check if starts empty:',np.shape(arr5))
#     ##arr5 = (arr4 >= 0)
#     #for n in arr4 != np.nan:
#     #    arr5.append(n)
#     ##diff = len(arr3) - len(arr5)
#     ##extras = np.repeat('False',diff)
#     ##print('extra info:',np.shape(extras),extras)
#     ##arr5 = np.append(arr5, extras)
#     #arr5 = np.array(arr5, dtype=object)
#     ##arr5 == 'True' 
#     ##print('shape after append:',np.shape(arr5),type(arr5),arr5)
#     newerarr[arr4] = arr3[arr4] #cant do this with a mask
#     mastermatch.append(newerarr)

In [23]:
# x = np.ma.array(np.arange(5), mask=[0]*2 + [1]*3)
# x.compressed()

# type(x.compressed())

In [24]:
# allfilters=[]
# for mask in masks:
#     eachfilter = mask[~np.isnan(mask)]
#     allfilters.append(eachfilter)
# print(np.shape(allfilters),allfilters[0])

In [25]:
# #testing how to match masks to masterdata size and then insert it into above loop before masks.append
# import numpy.ma as ma
# mastermatch = []
# for i in allfilters:
#     arr3 = np.arange(0,len(masterdata),1) #might wana start at 0 and +1 b/c recno goes from 1-89 but 4 now this is 0-88
#     arr4 = i.astype('int')
#     newerarr = np.zeros(len(arr3))
#     newerarr[:] = np.nan
#     #print('my test:',np.shape(arr2),np.shape(arr1))#,arr2)
#     newerarr[arr4] = arr3[arr4]
#     mastermatch.append(newerarr)
# print(np.shape(mastermatch),mastermatch[0])

In [26]:
# filters = ("F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP")

# masterkabooms = Table([mastermatch[0],mastermatch[1],mastermatch[2],mastermatch[3],mastermatch[4],\
#                        mastermatch[5],mastermatch[6],mastermatch[7],mastermatch[8],mastermatch[9]],\
#                       names=("F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP"))
# print(masterkabooms[:])

In [27]:
# for num,i in enumerate(masterdata['NUMBER_F606']):  ###this proves mask values are index of masterdata for each filter
#      print(num,i)

#T606

In [28]:
#checking = learned these wont match to masterdata yet, these are only masks per filter's detections!

# viewing = hstack(masks) #values in table is the # of object in filter, col are each filter, table length matches masterdata
# # for row in viewing:
# #     print(row)
# viewing
# # for i in viewing['col0_1']:
# #     print(i)

In [29]:
#trying to match these masks to full masterdata 


#newtable = masterdata['NUMBER_F105','NUMBER_F125','NUMBER_F435']
# ONLY = masterdata[np.where(masterdata['NUMBER_F105']>0)]
# ONLY

In [30]:
#mytables[0]#['detection_number']

In [31]:
# for i in np.where(masterdata['NUMBER_F435']>0):
#     print(i)

In [32]:
#test on single filter


#gets corresponding indices

# arr1 = (np.where(masterdata['NUMBER_F105'] > 0)[0])
# print(type(arr1),arr1,'lenght:',len(arr1))
# arr2 = T105['detection_number'].astype('int')
# print('t2',type(arr2),len(arr2))

# newarr = np.zeros(53)
# newarr[:] = np.nan
# newarr[arr2] = arr1[arr2]
# print(newarr) #these are the indices of masterdata table that have kaboom values for F105 detections

In [115]:
# # totalgals = np.zeros(len(masterdata))
# # np.shape(totalgals)
# # new = totalgals + [T105['detection_number'].data]
# # new
# values105 = T105['detection_number'].data
# print(type(values105),np.shape(values105),'problem',len((masterdata['NUMBER_F105']>0)),len(T105['detection_number'].data))
# total = len(masterdata['NUMBER_F105']>0)
# kbs = len(T105['detection_number'].data)
# missing = total - kbs
# print(type(missing),np.shape(missing),missing)
# new = np.append(values105, missing)
# np.shape(new)

In [112]:
# # m105 = join(masterdata['recno'],T105)
# # m105
# mask = np.where(masterdata['NUMBER_F105'] > 0)
# len(mask[0]),mask[0][::]
# #type(mask[0])
# totaldetections = np.arange(0,len(mask))
# mask #index in masterdata of detections (length 53)


In [113]:
# collect = []
# for i in master:
#     #print(i) #0-88
#     if i not in T105['detection_number'].data:
#         newi = 0
# #     elif i != np.any(T105['detection_number']:
# #         newi =1
#     collect.append(newi)
# len(collect)

In [114]:
# T105['detection_number'].data


In [209]:
# for count,value in enumerate(mask):
#     abc = (count == np.any(T105['detection_number']))
# abc,len(mask[0])
# b=[]
# for a in T105['detection_number']:
#     print('firsta:',a)
#     for i in totaldetections:
#         print('i:',i,'a:',a)
#         if i == a:
#             c = 0
#             b.append(c)
#         else:
#             c=1
#             b.append(c)

# len(b),b

In [109]:
# mask = []
# for f in filters:
#     numbers = []
#     print('one')
#     for row in kaboom:
#         master = np.arange(0,len(masterdata))
#         print('two')
#         if row['filter'] == f:
#             print('three')
#             nums = row['detection_number']
#             numbers.append(nums)
#     print('nums:',len(numbers))
#     mask = master[(np.where(numbers== np.any(master)))]
# #     for i in numbers:
# #         print('four')
# #         #print(type(i))
# #         master = master[(np.where(i == master))]
# #         mask.append(ma)
#     break
# mask

In [111]:
#trying to split table
# indexlist = np.arange(0,len(masterdata))
# print(np.shape(indexlist))

# allkaboomnums = []
# kaboomt = Table([indexlist,indexlist],names=('asecond4noreason','master_number'))
# for f in filters:
#     #if kaboom['filter'] == f:
#     split = np.where(kaboom['filter']==f)
#     lowerlim = np.min(split)
#     upperlim = np.max(split)
#     new = kaboom['detection_number'][lowerlim:upperlim+1]
#     allkaboomnums.append(new)
#     print(type(new),len(new))
#     kaboomt = hstack([kaboomt,new])#,uniq_col_name = '{col_name}'.format(f))
# kaboomt

# indexlist = np.arange(0,len(masterdata))
# repeatedfull = np.repeat([indexlist],10,axis=0)
# print(np.shape(repeatedfull),repeatedfull)

In [110]:
# for i,value in enumerate(allkabooms):
#     #print(allkabooms[i],'nexxti',value,'next')
#     values = np.array(allkabooms[i])
#     fn = 
#     kaboomt = hstack([kaboomt,values],uniq_col_name = '{}_{}'.format(fn,i))
# kaboomt

# getting example plots for presentation


In [3]:

def select_object_segmap(segmap,index):
    Regions = np.zeros_like(segmap)
    if index not in segmap:
        raise ValueError('index not in segmentation map')
    Regions[segmap==index]=index
    return Regions

In [7]:
def usingfilter(filtername):
    #filter = filtername
    detected_objs_file_table = ascii.read('/run/media/rowland/PURPLEDRIVE/testPSF_{}.cat'.format(filtername))
    image_file = pyfits.open('/run/media/rowland/PURPLEDRIVE/elgordo_{}.fits'.format(filtername))
    segmap_file = pyfits.open('/run/media/rowland/PURPLEDRIVE/segmaps_check/{}_check.fits'.format(filtername))
    data_perfilter = detected_objs_file_table, image_file, segmap_file, filtername
    return(data_perfilter)

In [8]:
filters = ["F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP","F105W_galfit_test_folder"]
zeropoints = {"F435W":25.665,"F606W":26.493,"F814W":25.97,"F105W":26.2687,"F125W":26.2303,"F140W":24.4524,\
              "F160W":25.9463, "F625W":24.85714771, "F775W":25.66195995, "F850LP":25.89943399}

#ONLY CHANGES NEEDED: FILTER HERE, ADJUST B1&B2 VALUES & MAG CUT AS NEEDED
############################################################################
data = usingfilter(filters[0])
############################################################################
print('Currently using this filter:',data[3])

Currently using this filter: F105W


In [9]:
detected_objs_file = data[0]
image_file = data[1]
segmap_file = data[2]
filtername = data[3]


ACT-CL,SMH,RAJ2000,DEJ2000,imag,z,e_z,rcc,MSF,Simbad,recno,NUMBER_F105,MAG_AUTO_F105,MAGERR_AUTO_F105,KRON_RADIUS_F105,ISOAREA_IMAGE_F105,XPEAK_IMAGE_F105,YPEAK_IMAGE_F105,X_IMAGE_F105,Y_IMAGE_F105,ALPHA_J2000_F105,DELTA_J2000_F105,A_IMAGE_F105,THETA_IMAGE_F105,MU_MAX_F105,ELLIPTICITY_F105,CLASS_STAR_F105,FLUX_RADIUS_F105,Separation_1,NUMBER_F125,MAG_AUTO_F125,MAGERR_AUTO_F125,KRON_RADIUS_F125,ISOAREA_IMAGE_F125,XPEAK_IMAGE_F125,YPEAK_IMAGE_F125,X_IMAGE_F125,Y_IMAGE_F125,ALPHA_J2000_F125,DELTA_J2000_F125,A_IMAGE_F125,THETA_IMAGE_F125,MU_MAX_F125,ELLIPTICITY_F125,CLASS_STAR_F125,FLUX_RADIUS_F125,Separation_1a,NUMBER_F140,MAG_AUTO_F140,MAGERR_AUTO_F140,KRON_RADIUS_F140,ISOAREA_IMAGE_F140,XPEAK_IMAGE_F140,YPEAK_IMAGE_F140,X_IMAGE_F140,Y_IMAGE_F140,ALPHA_J2000_F140,DELTA_J2000_F140,A_IMAGE_F140,THETA_IMAGE_F140,MU_MAX_F140,ELLIPTICITY_F140,CLASS_STAR_F140,FLUX_RADIUS_F140,Separation_1b,NUMBER_F160,MAG_AUTO_F160,MAGERR_AUTO_F160,KRON_RADIUS_F160,ISOAREA_IMAGE_F160,XPEAK_IMAGE_F160,YPEAK_IMAGE_F160,X_IMAGE_F160,Y_IMAGE_F160,ALPHA_J2000_F160,DELTA_J2000_F160,A_IMAGE_F160,THETA_IMAGE_F160,MU_MAX_F160,ELLIPTICITY_F160,CLASS_STAR_F160,FLUX_RADIUS_F160,Separation_1c,NUMBER_F435,MAG_AUTO_F435,MAGERR_AUTO_F435,KRON_RADIUS_F435,ISOAREA_IMAGE_F435,XPEAK_IMAGE_F435,YPEAK_IMAGE_F435,X_IMAGE_F435,Y_IMAGE_F435,ALPHA_J2000_F435,DELTA_J2000_F435,A_IMAGE_F435,THETA_IMAGE_F435,MU_MAX_F435,ELLIPTICITY_F435,CLASS_STAR_F435,FLUX_RADIUS_F435,Separation_1d,NUMBER_F606,MAG_AUTO_F606,MAGERR_AUTO_F606,KRON_RADIUS_F606,ISOAREA_IMAGE_F606,XPEAK_IMAGE_F606,YPEAK_IMAGE_F606,X_IMAGE_F606,Y_IMAGE_F606,ALPHA_J2000_F606,DELTA_J2000_F606,A_IMAGE_F606,THETA_IMAGE_F606,MU_MAX_F606,ELLIPTICITY_F606,CLASS_STAR_F606,FLUX_RADIUS_F606,Separation_1e,NUMBER_F625,MAG_AUTO_F625,MAGERR_AUTO_F625,KRON_RADIUS_F625,ISOAREA_IMAGE_F625,XPEAK_IMAGE_F625,YPEAK_IMAGE_F625,X_IMAGE_F625,Y_IMAGE_F625,ALPHA_J2000_F625,DELTA_J2000_F625,A_IMAGE_F625,THETA_IMAGE_F625,MU_MAX_F625,ELLIPTICITY_F625,CLASS_STAR_F625,FLUX_RADIUS_F625,Separation_1f,NUMBER_F775,MAG_AUTO_F775,MAGERR_AUTO_F775,KRON_RADIUS_F775,ISOAREA_IMAGE_F775,XPEAK_IMAGE_F775,YPEAK_IMAGE_F775,X_IMAGE_F775,Y_IMAGE_F775,ALPHA_J2000_F775,DELTA_J2000_F775,A_IMAGE_F775,THETA_IMAGE_F775,MU_MAX_F775,ELLIPTICITY_F775,CLASS_STAR_F775,FLUX_RADIUS_F775,Separation_1g,NUMBER_F814,MAG_AUTO_F814,MAGERR_AUTO_F814,KRON_RADIUS_F814,ISOAREA_IMAGE_F814,XPEAK_IMAGE_F814,YPEAK_IMAGE_F814,X_IMAGE_F814,Y_IMAGE_F814,ALPHA_J2000_F814,DELTA_J2000_F814,A_IMAGE_F814,THETA_IMAGE_F814,MU_MAX_F814,ELLIPTICITY_F814,CLASS_STAR_F814,FLUX_RADIUS_F814,Separation_1h,NUMBER_F850,MAG_AUTO_F850,MAGERR_AUTO_F850,KRON_RADIUS_F850,ISOAREA_IMAGE_F850,XPEAK_IMAGE_F850,YPEAK_IMAGE_F850,X_IMAGE_F850,Y_IMAGE_F850,ALPHA_J2000_F850,DELTA_J2000_F850,A_IMAGE_F850,THETA_IMAGE_F850,MU_MAX_F850,ELLIPTICITY_F850,CLASS_STAR_F850,FLUX_RADIUS_F850,Separation
str10,str18,float64,float64,float64,float64,float64,float64,str16,str6,int64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float6

In [14]:

segmap = segmap_file[0].data+0 #PARAMETER
collected_regions = []

for detection in masterdata: 
    xc = detection["X_IMAGE_F105"] #in pix #PARAMETER
    yc = detection["Y_IMAGE_F105"] #in pix #PARAMETER
    pixscale = 0.06 #PARAMETER
    index = detection['NUMBER_F105']
    print(type(index))
    #radius=0.5 ##DEFAULT value for reference #PARAMETER
    if type(index) == type(detection['NUMBER_F105'][0]:
        
        each_region = select_object_segmap(segmap,index) #indexing so have matching value so subtracting =0
        collected_regions.append(each_region)
    else:
        continue
collected_regions = np.array(collected_regions)
print('collected_regions: type-',type(collected_regions), 'shape-',collected_regions.shape)    
final_regions = np.sum(collected_regions, axis=0) #COLLECTING ALL OUTPUTS TO ONE MASK
print('shape of final regions:',final_regions.shape)

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.int64'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ma.core.MaskedConstant'>
<class 'numpy.int6

In [ ]:
#subtracting
mask_file = segmap - final_regions #so the galaxies of the cluster have value=0

#setting values to be binary, all none cluster galaxy pixels are =1 (so galfit will ignore)
mask_file[np.where(mask_file > 0)]=1
#setting regions outside of observation to =1
mask_file[image_data==0]=1

In [ ]:
fig, axis = mpl.subplots(1,4,figsize=(18,8),sharex=True,sharey=True)
axis[0].imshow(image_data,vmin = -0.01,vmax = 0.1)
axis[0].set_title('hubble data')
axis[1].imshow(segmap,vmin=0,vmax=1)
axis[1].set_title('segmap')
axis[2].imshow(final_regions,vmin=0, vmax=1)
axis[2].set_title('detected regions')
axis[3].imshow(mask_file,vmin=0,vmax=1)
axis[3].set_title('mask')